In [2]:
# BASE
import pandas as pd
import numpy as np
import sklearn
from sklearn import set_config
import pickle

#VISUALIZATION
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from sklearn.tree import plot_tree


#??????????????????????? WHAT ARE YOU????????????????????????
from scipy.stats import mode                                                        #?????????????????????????????????
from sklearn.datasets import load_digits                                            #?????????????????????????????
from sklearn.decomposition import PCA                                               #??????????????????????????
from sklearn.manifold import TSNE                                                   #????????????????????????????
from sklearn.datasets import make_classification                                    #??????????????????????????
from scipy.special import expit                                                     #???????????????????????????


#VOTING
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

from xgboost import XGBClassifier


#CLASIFIERS
from sklearn.tree import DecisionTreeClassifier                                     
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV                # search of the best params for random_forrest
from sklearn.linear_model import SGDClassifier                                      #????????????????????????
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors                                      #????????????????????????
from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC                                                         #???????????
from sklearn.ensemble import AdaBoostClassifier                                     #?????????????




#REGRESSORS
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor 


from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor





#CLASTERING
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
import scipy.cluster.hierarchy as sch                                               #????????????????????????????




# SCALERS and TRANSFORMATION
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import FunctionTransformer

from sklearn.pipeline import Pipeline                                               # pipeline function for transformers
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, KBinsDiscretizer
from sklearn.impute import SimpleImputer




# metrics and processing 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, roc_curve, roc_auc_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from sklearn.model_selection import train_test_split
from scipy.stats import zscore, boxcox
from sklearn.model_selection import cross_val_score                                 #??????????????????????????



# EDA data treatment
import missingno


# other (mainly system libs)
import warnings
import sys
from io import StringIO
import time
import os
import shutil
from dotenv import load_dotenv
from sqlalchemy import create_engine
import itertools
from collections import Counter
import urllib.request as req
import zipfile


#####################################################
# Constants
SEED = 50
URL1 = 'https://www.kaggle.com/api/v1/datasets/download/ulrikthygepedersen/kickstarter-projects'
URL2 = 'https://www.kaggle.com/api/v1/datasets/download/watts2/glove6b50dtxt'
################################################################

# Downloading data
req.urlretrieve(URL1, '/data/data.zip')
zipfile.ZipFile('/data/data.zip', 'a').extractall(path='./data/')
req.urlretrieve(URL2, '/data/glove.zip')
zipfile.ZipFile('/data/glove.zip', 'a').extractall(path='./data/')

# Processing downloads
df = pd.read_csv('./data/kickstarter_projects.csv')


df.columns = [val.strip().replace(' ','_').lower() for val in df.columns.tolist()]
df = df.drop('id', axis=1)

df = df[(df['state'] == 'Successful') | (df['state'] == 'Failed')]
df['state'] = df['state'].map({'Successful':1 , 'Failed': 0})

print(df.info())
print(df.describe())
# ## Splitting data for testing 
X_train, X_test, y_train, y_test = train_test_split(df.drop('state', axis=1), df.state, train_size=0.8, stratify=df.state, random_state=SEED)
X_train.to_csv('./data/X_train.csv',index=False)
X_test.to_csv('./data/X_test.csv',index=False)
y_train.to_csv('./data/y_train.csv',index=False)
y_test.to_csv('./data/y_test.csv',index=False)




<class 'pandas.core.frame.DataFrame'>
Index: 331462 entries, 0 to 374605
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   name         331462 non-null  object
 1   category     331462 non-null  object
 2   subcategory  331462 non-null  object
 3   country      331462 non-null  object
 4   launched     331462 non-null  object
 5   deadline     331462 non-null  object
 6   goal         331462 non-null  int64 
 7   pledged      331462 non-null  int64 
 8   backers      331462 non-null  int64 
 9   state        331462 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 27.8+ MB
None
               goal       pledged        backers          state
count  3.314620e+05  3.314620e+05  331462.000000  331462.000000
mean   4.152286e+04  9.939993e+03     116.456315       0.403820
std    1.109279e+06  9.664561e+04     965.732911       0.490663
min    0.000000e+00  0.000000e+00       0.000000       0.000000
25%    2.00000

In [54]:
# Loading splits for processing
X_train = pd.read_csv('./data/X_train.csv')
X_test = pd.read_csv('./data/X_test.csv')
y_train = pd.read_csv('./data/y_train.csv')
y_test = pd.read_csv('./data/y_test.csv')


def calc_dol_p_back(df:pd.DataFrame)-> pd.DataFrame:
    df['backers'] = df['backers'] + np.nextafter(0,1)
    df['log_dol_p_back'] = df['pledged']/df['backers']
    df['backers'] = df['backers'].astype('int') 
    return df

def max_frac(ds:pd.Series)-> pd.Series:
    if ds.max() != 0:
        ds = ds/ds.max()
    return ds

def log10p1_of_val(val):
    return np.log10(val+1)


def pre_processing(df:pd.DataFrame)-> pd.DataFrame:
# Formating dates
    df['launched'] = pd.to_datetime(df['launched'], format='%Y-%m-%d %H:%M:%S')
    df['deadline'] = pd.to_datetime(df['deadline'], format='%Y-%m-%d')
    df['lin_duration_days'] = df['deadline'] - X_train['launched']
    df['lin_duration_days'] = max_frac(df['lin_duration_days'].dt.days.astype('int'))


# Splitting ("dymming") and dumping launch dates:
#           LAUNCH DATE
    df['launched_day'] = df['launched'].dt.day
    df['launched_month'] = df['launched'].dt.month
    df['launched_year'] = df['launched'].dt.year
    df['launched_dow'] = df['launched'].dt.day_name()
    df = df.join(pd.get_dummies(df['launched_dow'], prefix='launched', prefix_sep='_', drop_first = False, dtype=float)).copy()
    least_common = pd.DataFrame(df['launched_dow'].value_counts(sort=True, ascending=True)).index.tolist()[0]
    df = df.drop('launched_'+least_common, axis=1)
    df = df.drop(['launched_dow', 'launched'], axis=1)
#           DEADLINE DATE
    df['deadline_day'] = df['deadline'].dt.day
    df['deadline_month'] = df['deadline'].dt.month
    df['deadline_year'] = df['deadline'].dt.year
    df['deadline_dow'] = df['deadline'].dt.day_name()
    df = df.join(pd.get_dummies(df['deadline_dow'], prefix='deadline', prefix_sep='_', drop_first = False, dtype=float)).copy()
    least_common = pd.DataFrame(df['deadline_dow'].value_counts(sort=True, ascending=True)).index.tolist()[0]
    df = df.drop('deadline_'+least_common, axis=1)
    df = df.drop(['deadline', 'deadline_dow'], axis=1)
    
# calculating dol_p_back:
    df = calc_dol_p_back(df)
  
# Logarithmizing
    df['goal_lin'] = max_frac(df['goal'])
    df['pledged_lin'] = max_frac(df['pledged'])
    df['backers_lin'] = max_frac(df['backers'])
#    df['pledged_log10'] = df['pledged'].apply(lambda x: log10p1_of_val(x))
#    df['backers_log10'] = df['backers'].apply(lambda x: log10p1_of_val(x))

    df = df.drop(['goal', 'pledged', 'backers'], axis=1).copy()


# Splitting ("dymming") and object columns:
    obs = []
    for val in df.drop('name', axis=1).columns.tolist():
        if df[val].dtype.name == 'object':
            obs.append(val)
    for val in obs:
        least_common = pd.DataFrame(df[val].value_counts(sort=True, ascending=True)).index.tolist()[0]
        df = df.join(pd.get_dummies(df[val], prefix=val, prefix_sep='_', drop_first = False, dtype=int))
        df = df.drop(val+'_'+least_common, axis=1)
    df = df.drop(obs, axis = 1)

    return df


X_train = pre_processing(X_train)




In [60]:

for val in X_train.columns.tolist():
    try:
        if X_train[val].max() > 1:
            print(val)
    except: 
        print(val, ' - ', X_train[val].dtype)

X_train['launched_day'].describe()


name  -  object
launched_day
launched_month
launched_year
deadline_day
deadline_month
deadline_year
log_dol_p_back


count    265169.000000
mean         15.280172
std           8.819738
min           1.000000
25%           8.000000
50%          15.000000
75%          23.000000
max          31.000000
Name: launched_day, dtype: float64